In [1]:
#!pip install pymc3
#!pip install pymc
#!pip install hmmlearn gensim
#!pip install econml

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Function to generate random touchpoint sequences
def generate_touchpoint_sequence():
    touchpoints = ['email', 'social', 'search', 'display', 'video', 'affiliate']
    sequence_length = np.random.randint(1, 6)
    return np.random.choice(touchpoints, size=sequence_length, replace=True)

# Generate data
np.random.seed(42)
n_users = 1000
max_touchpoints_per_user = 5

data = []
start_date = datetime(2023, 1, 1)

for user_id in range(1, n_users + 1):
    n_touchpoints = np.random.randint(1, max_touchpoints_per_user + 1)
    touchpoints = generate_touchpoint_sequence()
    
    for i, touchpoint in enumerate(touchpoints):
        timestamp = start_date + timedelta(days=np.random.randint(0, 365))
        conversion = 1 if i == len(touchpoints) - 1 and np.random.random() < 0.2 else 0
        data.append([user_id, touchpoint, timestamp, conversion])

df = pd.DataFrame(data, columns=['user_id', 'touchpoint', 'timestamp', 'conversion'])
df = df.sort_values(['user_id', 'timestamp']).reset_index(drop=True)

print(f"Total number of data points: {len(df)}")
print(df.head(10))

Total number of data points: 3024
   user_id touchpoint  timestamp  conversion
0        1      video 2023-03-29           0
1        1     social 2023-04-10           0
2        1     search 2023-08-03           0
3        1      video 2023-11-27           0
4        1     search 2023-12-26           0
5        2  affiliate 2023-07-11           0
6        2    display 2023-10-21           0
7        3  affiliate 2023-02-18           0
8        3      video 2023-02-28           0
9        3    display 2023-06-19           1


In [12]:
# Helper function to calculate attribution
def calculate_attribution(attribution_dict):
    total = sum(attribution_dict.values())
    return {channel: value / total for channel, value in attribution_dict.items()}

# 1. First-Touch Attribution Model
def first_touch_attribution(df):
    first_touches = df.groupby('user_id').first()['touchpoint'].value_counts()
    return calculate_attribution(first_touches.to_dict())

# 2. Last-Touch Attribution Model
def last_touch_attribution(df):
    last_touches = df.groupby('user_id').last()['touchpoint'].value_counts()
    return calculate_attribution(last_touches.to_dict())

# 3. Linear Attribution Model
def linear_attribution(df):
    all_touches = df['touchpoint'].value_counts()
    return calculate_attribution(all_touches.to_dict())

# Calculate attributions
first_touch_attr = first_touch_attribution(df)
last_touch_attr = last_touch_attribution(df)
linear_attr = linear_attribution(df)

# Print results
print("\nFirst-Touch Attribution:")
print(first_touch_attr)

print("\nLast-Touch Attribution:")
print(last_touch_attr)

print("\nLinear Attribution:")
print(linear_attr)

# Comparison of attributions
print("\nComparison of Attributions:")
all_channels = set(first_touch_attr.keys()) | set(last_touch_attr.keys()) | set(linear_attr.keys())

print(f"{'Channel':<10} {'First-Touch':<15} {'Last-Touch':<15} {'Linear':<15}")
print("-" * 55)
for channel in all_channels:
    first_value = first_touch_attr.get(channel, 0)
    last_value = last_touch_attr.get(channel, 0)
    linear_value = linear_attr.get(channel, 0)
    print(f"{channel:<10} {first_value:<15.4f} {last_value:<15.4f} {linear_value:<15.4f}")

# Conversion analysis
print("\nConversion Analysis:")
conversions = df[df['conversion'] == 1]
print(f"Total conversions: {len(conversions)}")
print("\nConversions by channel (Last-Touch):")
print(conversions['touchpoint'].value_counts(normalize=True))


First-Touch Attribution:
{'display': 0.183, 'video': 0.181, 'affiliate': 0.166, 'social': 0.16, 'email': 0.159, 'search': 0.151}

Last-Touch Attribution:
{'search': 0.182, 'email': 0.176, 'social': 0.172, 'display': 0.171, 'video': 0.159, 'affiliate': 0.14}

Linear Attribution:
{'video': 0.17592592592592593, 'display': 0.16832010582010581, 'search': 0.167989417989418, 'email': 0.16567460317460317, 'social': 0.16468253968253968, 'affiliate': 0.1574074074074074}

Comparison of Attributions:
Channel    First-Touch     Last-Touch      Linear         
-------------------------------------------------------
affiliate  0.1660          0.1400          0.1574         
display    0.1830          0.1710          0.1683         
search     0.1510          0.1820          0.1680         
social     0.1600          0.1720          0.1647         
video      0.1810          0.1590          0.1759         
email      0.1590          0.1760          0.1657         

Conversion Analysis:
Total conversi

In [13]:
# Helper function to calculate attribution
def calculate_attribution(attribution_dict):
    total = sum(attribution_dict.values())
    return {channel: value / total for channel, value in attribution_dict.items()}

# U-shaped Attribution Model
def u_shaped_attribution(df):
    attribution = {}
    for _, user_journey in df.groupby('user_id'):
        journey_length = len(user_journey)
        if journey_length == 1:
            touchpoint = user_journey['touchpoint'].iloc[0]
            attribution[touchpoint] = attribution.get(touchpoint, 0) + 1
        else:
            first_touch = user_journey['touchpoint'].iloc[0]
            last_touch = user_journey['touchpoint'].iloc[-1]
            attribution[first_touch] = attribution.get(first_touch, 0) + 0.4
            attribution[last_touch] = attribution.get(last_touch, 0) + 0.4
            
            middle_weight = 0.2 / (journey_length - 2) if journey_length > 2 else 0
            for touchpoint in user_journey['touchpoint'].iloc[1:-1]:
                attribution[touchpoint] = attribution.get(touchpoint, 0) + middle_weight
    
    return calculate_attribution(attribution)

# W-shaped Attribution Model
def w_shaped_attribution(df):
    attribution = {}
    for _, user_journey in df.groupby('user_id'):
        journey_length = len(user_journey)
        if journey_length == 1:
            touchpoint = user_journey['touchpoint'].iloc[0]
            attribution[touchpoint] = attribution.get(touchpoint, 0) + 1
        elif journey_length == 2:
            first_touch = user_journey['touchpoint'].iloc[0]
            last_touch = user_journey['touchpoint'].iloc[-1]
            attribution[first_touch] = attribution.get(first_touch, 0) + 0.5
            attribution[last_touch] = attribution.get(last_touch, 0) + 0.5
        else:
            first_touch = user_journey['touchpoint'].iloc[0]
            middle_touch = user_journey['touchpoint'].iloc[len(user_journey) // 2]
            last_touch = user_journey['touchpoint'].iloc[-1]
            attribution[first_touch] = attribution.get(first_touch, 0) + 0.3
            attribution[middle_touch] = attribution.get(middle_touch, 0) + 0.3
            attribution[last_touch] = attribution.get(last_touch, 0) + 0.3
            
            other_weight = 0.1 / (journey_length - 3) if journey_length > 3 else 0
            for i, touchpoint in enumerate(user_journey['touchpoint']):
                if i not in [0, len(user_journey) // 2, len(user_journey) - 1]:
                    attribution[touchpoint] = attribution.get(touchpoint, 0) + other_weight
    
    return calculate_attribution(attribution)

# Calculate attributions
u_shaped_attr = u_shaped_attribution(df)
w_shaped_attr = w_shaped_attribution(df)

# Print results
print("\nU-shaped Attribution:")
print(u_shaped_attr)

print("\nW-shaped Attribution:")
print(w_shaped_attr)

# Comparison of attributions
print("\nComparison of Attributions:")
all_channels = set(u_shaped_attr.keys()) | set(w_shaped_attr.keys())

print(f"{'Channel':<10} {'U-shaped':<15} {'W-shaped':<15}")
print("-" * 40)
for channel in all_channels:
    u_value = u_shaped_attr.get(channel, 0)
    w_value = w_shaped_attr.get(channel, 0)
    print(f"{channel:<10} {u_value:<15.4f} {w_value:<15.4f}")

# Conversion analysis
print("\nConversion Analysis:")
conversions = df[df['conversion'] == 1]
print(f"Total conversions: {len(conversions)}")
print("\nConversions by channel (Last-Touch):")
print(conversions['touchpoint'].value_counts(normalize=True))


U-shaped Attribution:
{'video': 0.1728767693588675, 'search': 0.16319733555370539, 'social': 0.16656258673327784, 'affiliate': 0.15233832917013607, 'display': 0.17703996669442118, 'email': 0.16798501248959208}

W-shaped Attribution:
{'video': 0.17322794492605834, 'search': 0.16195818459969394, 'social': 0.16715961244263117, 'affiliate': 0.1518612952575213, 'display': 0.17812340642529348, 'email': 0.16766955634880182}

Comparison of Attributions:
Channel    U-shaped        W-shaped       
----------------------------------------
affiliate  0.1523          0.1519         
display    0.1770          0.1781         
search     0.1632          0.1620         
social     0.1666          0.1672         
video      0.1729          0.1732         
email      0.1680          0.1677         

Conversion Analysis:
Total conversions: 217

Conversions by channel (Last-Touch):
video        0.225806
social       0.188940
search       0.175115
email        0.152074
affiliate    0.138249
display      0

In [15]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Prepare data for logistic regression
def prepare_data(df):
    # Create a feature matrix for each user journey
    user_journeys = df.groupby('user_id')['touchpoint'].apply(list).reset_index()
    user_journeys['conversion'] = df.groupby('user_id')['conversion'].last().values
    
    # One-hot encode the touchpoints
    touchpoints = ['email', 'social', 'search', 'display', 'video', 'affiliate']
    feature_matrix = np.zeros((len(user_journeys), len(touchpoints)))
    
    for idx, journey in enumerate(user_journeys['touchpoint']):
        for touchpoint in journey:
            feature_matrix[idx, touchpoints.index(touchpoint)] = 1
    
    return feature_matrix, user_journeys['conversion'].values

# Data-Driven Attribution Model
def data_driven_attribution(df):
    X, y = prepare_data(df)
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Train logistic regression model
    model = LogisticRegression(random_state=42)
    model.fit(X_train, y_train)
    
    # Get feature importance (coefficients)
    importance = model.coef_[0]
    
    # Create attribution dictionary
    touchpoints = ['email', 'social', 'search', 'display', 'video', 'affiliate']
    attribution = dict(zip(touchpoints, importance))
    
    # Normalize attribution scores
    total = sum(abs(value) for value in attribution.values())
    attribution = {channel: abs(value) / total for channel, value in attribution.items()}
    
    return attribution, model.score(X_test, y_test)

# Calculate data-driven attribution
data_driven_attr, model_accuracy = data_driven_attribution(df)

# Print results
print("\nData-Driven Attribution:")
for channel, score in data_driven_attr.items():
    print(f"{channel}: {score:.4f}")

print(f"\nModel Accuracy: {model_accuracy:.4f}")

# Comparison with other models (assuming you've calculated these in previous steps)
# If not, you can remove or comment out this part
print("\nComparison of Attributions:")
all_channels = set(data_driven_attr.keys())

print(f"{'Channel':<10} {'Data-Driven':<15}")
print("-" * 25)
for channel in all_channels:
    dd_value = data_driven_attr.get(channel, 0)
    print(f"{channel:<10} {dd_value:<15.4f}")

# Additional analysis
print("\nChannel Importance Ranking:")
sorted_channels = sorted(data_driven_attr.items(), key=lambda x: x[1], reverse=True)
for channel, importance in sorted_channels:
    print(f"{channel}: {importance:.4f}")

# Conversion analysis
print("\nConversion Analysis:")
conversions = df[df['conversion'] == 1]
print(f"Total conversions: {len(conversions)}")
print("\nConversions by channel (Last-Touch):")
print(conversions['touchpoint'].value_counts(normalize=True))


Data-Driven Attribution:
email: 0.1062
social: 0.1946
search: 0.1927
display: 0.2067
video: 0.1267
affiliate: 0.1732

Model Accuracy: 0.8950

Comparison of Attributions:
Channel    Data-Driven    
-------------------------
affiliate  0.1732         
search     0.1927         
display    0.2067         
social     0.1946         
video      0.1267         
email      0.1062         

Channel Importance Ranking:
display: 0.2067
social: 0.1946
search: 0.1927
affiliate: 0.1732
video: 0.1267
email: 0.1062

Conversion Analysis:
Total conversions: 217

Conversions by channel (Last-Touch):
video        0.225806
social       0.188940
search       0.175115
email        0.152074
affiliate    0.138249
display      0.119816
Name: touchpoint, dtype: float64


# Machine Learning Based MTA models

In [3]:
from scipy.stats import beta
from scipy.special import expit

def bayesian_attribution(df, n_iterations=1000):
    touchpoints = df['touchpoint'].unique()
    X = pd.get_dummies(df['touchpoint']).values
    y = df['conversion'].values

    n_features = X.shape[1]
    
    # Initialize parameters
    beta_coeffs = np.zeros(n_features)
    
    # Prior parameters
    alpha_prior = 1.0
    beta_prior = 1.0
    
    # MCMC sampling
    samples = np.zeros((n_iterations, n_features))
    
    for i in range(n_iterations):
        # Calculate probabilities
        p = expit(X.dot(beta_coeffs))
        
        # Sample new coefficients
        for j in range(n_features):
            likelihood = np.prod(p[X[:, j] == 1] ** y[X[:, j] == 1] * (1 - p[X[:, j] == 1]) ** (1 - y[X[:, j] == 1]))
            prior = beta.pdf(expit(beta_coeffs[j]), alpha_prior, beta_prior)
            posterior = likelihood * prior
            
            # Metropolis-Hastings step
            proposal = beta_coeffs[j] + np.random.normal(0, 0.1)
            beta_coeffs_proposal = beta_coeffs.copy()
            beta_coeffs_proposal[j] = proposal
            p_proposal = expit(X.dot(beta_coeffs_proposal))
            likelihood_proposal = np.prod(p_proposal[X[:, j] == 1] ** y[X[:, j] == 1] * (1 - p_proposal[X[:, j] == 1]) ** (1 - y[X[:, j] == 1]))
            prior_proposal = beta.pdf(expit(proposal), alpha_prior, beta_prior)
            posterior_proposal = likelihood_proposal * prior_proposal
            
            if np.random.random() < min(1, posterior_proposal / posterior):
                beta_coeffs[j] = proposal
        
        samples[i] = beta_coeffs
    
    # Calculate attribution
    attribution = {}
    for i, channel in enumerate(touchpoints):
        attribution[channel] = np.mean(samples[:, i])

    # Normalize attribution
    total = sum(attribution.values())
    attribution = {channel: value / total for channel, value in attribution.items()}

    return attribution

print("\nBayesian Attribution:")
print(bayesian_attribution(df))


Bayesian Attribution:
{'video': 0.17486857757843274, 'social': 0.18844845363975676, 'search': 0.17290025392415326, 'affiliate': 0.1617777383007844, 'display': 0.1542658800099353, 'email': 0.14773909654693762}


In [4]:
def markov_chain_attribution(df):
    # Create transition matrix
    touchpoints = df['touchpoint'].unique()
    n = len(touchpoints)
    transition_matrix = np.zeros((n+1, n+1))
    
    for _, user_data in df.groupby('user_id'):
        touchpoint_sequence = user_data['touchpoint'].tolist()
        for i in range(len(touchpoint_sequence)):
            if i == len(touchpoint_sequence) - 1:
                if user_data['conversion'].iloc[-1] == 1:
                    transition_matrix[touchpoints.tolist().index(touchpoint_sequence[i])][n] += 1
            else:
                transition_matrix[touchpoints.tolist().index(touchpoint_sequence[i])][touchpoints.tolist().index(touchpoint_sequence[i+1])] += 1
    
    # Normalize transition matrix
    row_sums = transition_matrix.sum(axis=1)
    transition_matrix = np.divide(transition_matrix, row_sums[:, np.newaxis], where=row_sums[:, np.newaxis]!=0)
    
    # Calculate removal effect
    base_conversion = transition_matrix[:n, n].sum()
    attribution = {}
    for i, channel in enumerate(touchpoints):
        temp_matrix = transition_matrix.copy()
        temp_matrix[i, :] = 0
        temp_matrix[i, i] = 1
        temp_conversion = np.linalg.matrix_power(temp_matrix, 100)[:n, n].sum()
        attribution[channel] = base_conversion - temp_conversion
    
    # Normalize attribution
    total_attribution = sum(attribution.values())
    for channel in attribution:
        attribution[channel] /= total_attribution
    
    return attribution

print("Markov Chain Attribution:")
print(markov_chain_attribution(df))

Markov Chain Attribution:
{'video': 0.16666666668406133, 'social': 0.16666666666493796, 'search': 0.16666666668301616, 'affiliate': 0.16666666663883137, 'display': 0.1666666666532864, 'email': 0.16666666667586677}


In [5]:
import networkx as nx

def graph_attribution(df):
    G = nx.DiGraph()
    
    for _, user_data in df.groupby('user_id'):
        touchpoints = user_data['touchpoint'].tolist()
        for i in range(len(touchpoints) - 1):
            if G.has_edge(touchpoints[i], touchpoints[i+1]):
                G[touchpoints[i]][touchpoints[i+1]]['weight'] += 1
            else:
                G.add_edge(touchpoints[i], touchpoints[i+1], weight=1)
    
    pagerank = nx.pagerank(G, weight='weight')
    
    # Normalize attribution
    total = sum(pagerank.values())
    attribution = {channel: pagerank.get(channel, 0) / total for channel in df['touchpoint'].unique()}
    
    return attribution

print("\nGraph-Based (PageRank) Attribution:")
print(graph_attribution(df))


Graph-Based (PageRank) Attribution:
{'video': 0.17221980828053013, 'social': 0.16649305130743472, 'search': 0.17501687656568984, 'affiliate': 0.15545571884425657, 'display': 0.1624278933737435, 'email': 0.16838665162834526}


In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def create_rnn_attention_model(vocab_size, max_length):
    inputs = Input(shape=(max_length,))
    embedding = Embedding(vocab_size, 32)(inputs)
    lstm = LSTM(64, return_sequences=True)(embedding)
    attention = Attention()([lstm, lstm])
    pooling = GlobalAveragePooling1D()(attention)
    output = Dense(1, activation='sigmoid')(pooling)
    model = tf.keras.Model(inputs=inputs, outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Prepare data
le = LabelEncoder()
df['touchpoint_encoded'] = le.fit_transform(df['touchpoint'])

max_length = df.groupby('user_id')['touchpoint'].count().max()

X = df.groupby('user_id')['touchpoint_encoded'].apply(lambda x: x.tolist() + [0] * (max_length - len(x))).tolist()
y = df.groupby('user_id')['conversion'].last().tolist()

X = np.array(X)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train model
model = create_rnn_attention_model(len(le.classes_), max_length)
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=0)

# Get attribution
attribution = {}
for channel in df['touchpoint'].unique():
    channel_input = np.full((1, max_length), le.transform([channel])[0])
    attribution[channel] = model.predict(channel_input)[0][0]

# Normalize attribution
total = sum(attribution.values())
attribution = {channel: value / total for channel, value in attribution.items()}

print("\nDeep Learning (RNN with Attention) Attribution:")
print(attribution)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step

Deep Learning (RNN with Attention) Attribution:
{'video': 0.10976741282191595, 'social': 0.047988214775738246, 'search': 0.06218911583340448, 'affiliate': 0.6177546376009803, 'display': 0.04049652473778955, 'email': 0.12180409423017147}


In [7]:
from itertools import combinations

def shapley_value_attribution(df):
    touchpoints = df['touchpoint'].unique()
    
    def value(coalition):
        if len(coalition) == 0:
            return 0
        mask = df['touchpoint'].isin(coalition)
        return df.loc[mask, 'conversion'].sum() / df.loc[mask, 'user_id'].nunique()
    
    n = len(touchpoints)
    attribution = {channel: 0 for channel in touchpoints}
    
    for channel in touchpoints:
        for r in range(n):
            subsets = combinations(set(touchpoints) - {channel}, r)
            for subset in subsets:
                coalition = set(subset)
                margin = value(coalition | {channel}) - value(coalition)
                attribution[channel] += margin / (n * len(list(combinations(touchpoints, r))))
    
    # Normalize attribution
    total = sum(attribution.values())
    attribution = {channel: value / total for channel, value in attribution.items()}
    
    return attribution

print("\nShapley Value Attribution:")
print(shapley_value_attribution(df))


Shapley Value Attribution:
{'video': 0.24131463198474504, 'social': 0.2002106074213146, 'search': 0.18467668081806618, 'affiliate': 0.13679323568915447, 'display': 0.09147009914102003, 'email': 0.14553474494569962}


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score


# Prepare the data
le = LabelEncoder()
df['touchpoint_encoded'] = le.fit_transform(df['touchpoint'])

# Create features
def create_features(group):
    sequence = group['touchpoint_encoded'].tolist()
    time_diff = (group['timestamp'].max() - group['timestamp'].min()).total_seconds() / 3600  # in hours
    last_conversion = group['conversion'].iloc[-1]
    return pd.Series({
        'sequence': sequence,
        'time_diff': time_diff,
        'conversion': last_conversion
    })

features = df.groupby('user_id').apply(create_features)

# Pad sequences to have the same length
max_length = features['sequence'].apply(len).max()
X = np.array([seq + [0] * (max_length - len(seq)) for seq in features['sequence']])

# Add time difference as a feature
X = np.column_stack([X, features['time_diff'].values])

y = features['conversion'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to calculate attribution
def calculate_attribution(model, feature_importances):
    attribution = {}
    channel_importances = feature_importances[:len(le.classes_)]
    for channel, importance in zip(le.classes_, channel_importances):
        attribution[channel] = importance
    
    # Normalize attribution
    total = sum(attribution.values())
    attribution = {channel: value / total for channel, value in attribution.items()}
    return attribution

# Function to evaluate model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy:.4f}")

# Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

print("\nRandom Forest Attribution:")
rf_attribution = calculate_attribution(rf_model, rf_model.feature_importances_)
print(rf_attribution)
evaluate_model(rf_model, X_test, y_test)

# Gradient Boosting Machine
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

print("\nGradient Boosting Machine Attribution:")
gb_attribution = calculate_attribution(gb_model, gb_model.feature_importances_)
print(gb_attribution)
evaluate_model(gb_model, X_test, y_test)

# Ensemble (Simple average of attributions)
print("\nEnsemble Attribution:")
ensemble_attribution = {}
for channel in le.classes_:
    ensemble_attribution[channel] = (rf_attribution[channel] + gb_attribution[channel]) / 2

# Normalize ensemble attribution
total = sum(ensemble_attribution.values())
ensemble_attribution = {channel: value / total for channel, value in ensemble_attribution.items()}
print(ensemble_attribution)

# Evaluate ensemble (using simple voting)
ensemble_predictions = (rf_model.predict_proba(X_test)[:, 1] + gb_model.predict_proba(X_test)[:, 1]) / 2
ensemble_predictions = (ensemble_predictions > 0.5).astype(int)
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Model Accuracy: {ensemble_accuracy:.4f}")


Random Forest Attribution:
{'affiliate': 0.16194113131979124, 'display': 0.14408723096688394, 'email': 0.10748276195494735, 'search': 0.05636225186431357, 'social': 0.023559214269252563, 'video': 0.5065674096248114}
Model Accuracy: 0.8850

Gradient Boosting Machine Attribution:
{'affiliate': 0.12338730105804087, 'display': 0.09719127796331732, 'email': 0.15222611068950972, 'search': 0.056922175908909, 'social': 0.01898387568561863, 'video': 0.5512892586946044}
Model Accuracy: 0.8950

Ensemble Attribution:
{'affiliate': 0.14266421618891606, 'display': 0.12063925446510063, 'email': 0.12985443632222854, 'search': 0.05664221388661128, 'social': 0.021271544977435594, 'video': 0.528928334159708}
Ensemble Model Accuracy: 0.8900


In [9]:
from sklearn.preprocessing import LabelEncoder
from hmmlearn import hmm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from collections import defaultdict


print(f"Total number of data points: {len(df)}")

# Prepare the data
le = LabelEncoder()
df['touchpoint_encoded'] = le.fit_transform(df['touchpoint'])

# 1. Hidden Markov Model (HMM)
print("\nHidden Markov Model Attribution:")

# Prepare sequences for HMM
sequences = df.groupby('user_id')['touchpoint_encoded'].apply(list).tolist()
lengths = [len(seq) for seq in sequences]
X = np.concatenate(sequences).reshape(-1, 1)

# Print diagnostic information
print(f"Number of unique touchpoints: {len(le.classes_)}")
print(f"Shape of X: {X.shape}")

# Train HMM
n_components = min(3, len(le.classes_))  # Ensure n_components is not greater than number of touchpoints
model = hmm.MultinomialHMM(n_components=n_components, n_iter=100, random_state=42)
model.fit(X, lengths)

# Calculate attribution
transition_matrix = model.transmat_
emission_matrix = model.emissionprob_

print(f"Shape of emission_matrix: {emission_matrix.shape}")

hmm_attribution = {}
for i, touchpoint in enumerate(le.classes_):
    if i < emission_matrix.shape[1]:
        importance = np.mean(emission_matrix[:, i])
        hmm_attribution[touchpoint] = importance
    else:
        print(f"Warning: Touchpoint {touchpoint} (index {i}) not covered by emission matrix")
        hmm_attribution[touchpoint] = 0



# Normalize attribution
total = sum(hmm_attribution.values())
hmm_attribution = {channel: value / total for channel, value in hmm_attribution.items()}
print(hmm_attribution)

# 2. Latent Dirichlet Allocation (LDA) using scikit-learn
print("\nLatent Dirichlet Allocation Attribution:")

# Prepare documents for LDA
documents = df.groupby('user_id')['touchpoint'].apply(lambda x: ' '.join(x)).tolist()

# Create document-term matrix
vectorizer = CountVectorizer()
doc_term_matrix = vectorizer.fit_transform(documents)

# Train LDA model
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_output = lda_model.fit_transform(doc_term_matrix)

# Calculate attribution
feature_names = vectorizer.get_feature_names_out()
topic_term_dist = lda_model.components_ / lda_model.components_.sum(axis=1)[:, np.newaxis]

lda_attribution = defaultdict(float)
for topic_idx, topic in enumerate(topic_term_dist):
    for i, value in enumerate(topic):
        lda_attribution[feature_names[i]] += value

# Normalize attribution
total = sum(lda_attribution.values())
lda_attribution = {channel: value / total for channel, value in lda_attribution.items()}
print(dict(lda_attribution))

# Comparison of attributions
print("\nComparison of Attributions:")
all_channels = set(hmm_attribution.keys()) | set(lda_attribution.keys())

print(f"{'Channel':<10} {'HMM':<10} {'LDA':<10}")
print("-" * 30)
for channel in all_channels:
    hmm_value = hmm_attribution.get(channel, 0)
    lda_value = lda_attribution.get(channel, 0)
    print(f"{channel:<10} {hmm_value:<10.4f} {lda_value:<10.4f}")

MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Total number of data points: 3024

Hidden Markov Model Attribution:
Number of unique touchpoints: 6
Shape of X: (3024, 1)
Shape of emission_matrix: (3, 1)
{'affiliate': nan, 'display': nan, 'email': nan, 'search': nan, 'social': nan, 'video': nan}

Latent Dirichlet Allocation Attribution:
{'affiliate': 0.1434880786701127, 'display': 0.2003766555038235, 'email': 0.16665839337524785, 'search': 0.13999817723706937, 'social': 0.18400263317100113, 'video': 0.1654760620427454}

Comparison of Attributions:
Channel    HMM        LDA       
------------------------------
affiliate  nan        0.1435    
display    nan        0.2004    
search     nan        0.1400    
social     nan        0.1840    
video      nan        0.1655    
email      nan        0.1667    


In [11]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Prepare the data
le = LabelEncoder()
df['touchpoint_encoded'] = le.fit_transform(df['touchpoint'])

# Create features
ohe = OneHotEncoder(sparse_output=False)
X = ohe.fit_transform(df[['touchpoint']])
feature_names = ohe.get_feature_names_out(['touchpoint'])

# Create treatment and outcome variables
T = df['touchpoint_encoded'].values
y = df['conversion'].values

# Create some additional features
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month
W = df[['day_of_week', 'month']].values

# Split the data
X_train, X_test, T_train, T_test, y_train, y_test, W_train, W_test = train_test_split(X, T, y, W, test_size=0.2, random_state=42)

# 1. Simplified Causal Forest
print("\nSimplified Causal Forest Attribution:")

def simplified_causal_forest(X, T, y, W):
    # Train a random forest to predict treatment
    rf_t = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
    rf_t.fit(W, T)
    
    # Get residuals for treatment
    T_pred = rf_t.predict(W)
    T_residual = T - T_pred
    
    # Train a random forest to predict outcome
    rf_y = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
    rf_y.fit(W, y)
    
    # Get residuals for outcome
    y_pred = rf_y.predict(W)
    y_residual = y - y_pred
    
    # Final stage: regress outcome residuals on treatment residuals
    final_model = LinearRegression()
    final_model.fit(T_residual.reshape(-1, 1), y_residual)
    
    return final_model.coef_[0]

cf_effects = simplified_causal_forest(X_train, T_train, y_train, W_train)

cf_attribution = {}
for i, touchpoint in enumerate(le.classes_):
    cf_attribution[touchpoint] = abs(cf_effects)

# Normalize attribution
total = sum(cf_attribution.values())
cf_attribution = {channel: value / total for channel, value in cf_attribution.items()}
print(cf_attribution)

# 2. Simplified Double Machine Learning
print("\nSimplified Double Machine Learning Attribution:")

def simplified_double_ml(X, T, y, W):
    # First stage: predict T using W
    model_t = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
    model_t.fit(W, T)
    T_pred = model_t.predict(W)
    T_residual = T - T_pred

    # First stage: predict y using W
    model_y = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
    model_y.fit(W, y)
    y_pred = model_y.predict(W)
    y_residual = y - y_pred

    # Second stage: regress y_residual on T_residual
    final_model = LinearRegression()
    final_model.fit(T_residual.reshape(-1, 1), y_residual)
    
    return final_model.coef_[0]

dml_effects = simplified_double_ml(X_train, T_train, y_train, W_train)

dml_attribution = {}
for i, touchpoint in enumerate(le.classes_):
    dml_attribution[touchpoint] = abs(dml_effects)

# Normalize attribution
total = sum(dml_attribution.values())
dml_attribution = {channel: value / total for channel, value in dml_attribution.items()}
print(dml_attribution)

# Comparison of attributions
print("\nComparison of Attributions:")
all_channels = set(cf_attribution.keys()) | set(dml_attribution.keys())

print(f"{'Channel':<10} {'Causal Forest':<15} {'Double ML':<15}")
print("-" * 40)
for channel in all_channels:
    cf_value = cf_attribution.get(channel, 0)
    dml_value = dml_attribution.get(channel, 0)
    print(f"{channel:<10} {cf_value:<15.4f} {dml_value:<15.4f}")


Simplified Causal Forest Attribution:
{'affiliate': 0.16666666666666663, 'display': 0.16666666666666663, 'email': 0.16666666666666663, 'search': 0.16666666666666663, 'social': 0.16666666666666663, 'video': 0.16666666666666663}

Simplified Double Machine Learning Attribution:
{'affiliate': 0.16666666666666663, 'display': 0.16666666666666663, 'email': 0.16666666666666663, 'search': 0.16666666666666663, 'social': 0.16666666666666663, 'video': 0.16666666666666663}

Comparison of Attributions:
Channel    Causal Forest   Double ML      
----------------------------------------
affiliate  0.1667          0.1667         
display    0.1667          0.1667         
search     0.1667          0.1667         
social     0.1667          0.1667         
video      0.1667          0.1667         
email      0.1667          0.1667         
